# Actor support example

This notebook exhibits the support for multiple actors in one notebook which is facilitated by `decActors.pl`. We wish to be able to model the expectations of different agents given general events and fluents which are shared between all agents, as well as individualised events and fluents which apply to only one or a few of the agents.

### Set up

In [1]:
?- cd('~/work'), ['decActors'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

We declare four hospital agents (Otago, Christchurch, Wellington and Auckland) as exisiting initially. They are declared with the *support* agent argument, which is used to declare system wide facts which support other fluents.

In [2]:
% File: hospitals.pl
initially(support, hospital("Otago")).
initially(support, hospital("Christchurch")).
initially(support, hospital("Wellington")).
initially(support, hospital("Auckland")).


We declare that the *applyGeneral/1* fluent is true when its argument is a current hospital. This will allow us to apply fluents to all current hospitals. The *applyGeneral* fluent should not be used for events, as this is covered by our *happensAtNarrative* rule below, which states that narrative events are inherited by the descendants of the actor that the event occurs for. We can declare which agents are children of which other agents using the *inherits* fluent, and we declare that any current hospital inherits events from the *general* actor. We use the *general* actor to declare events that we want to occur for all current hospitals.

In [3]:
% File: generalisations.pl
% Set suitable agents to pass general fluent rules onto. Should not be used for events!
holdsAt(support, applyGeneral(Agent), T):- holdsAt(support, hospital(Agent),T).
% Hospitals inherit general events
holdsAt(support, inherits(Child, general),T):- holdsAt(support, hospital(Child),T).
% Narrative events also occur in the children of the responsible parent.
happensAtNarrative(Agent:Event,T) :- holdsAt(support, inherits(Agent,Parent),T), happensAtNarrative(Parent:Event,T).

We declare some test rules below in order to check that events and fluents are applying to the correct actors. 

We declare that for any current hospital, *lightOn* is true if *greenLight* is true, and set *greenLight* to be true for all active hospitals at time zero. We declare that for any hospital, *turnOff* terminates *greenLight*, and that for Otago Hospital only, *powerCut* terminates *greenLight*. 

An expectation rule holds initially for Otago Hospital only, which states that the hospital expects *redLight* will be true within 3 time periods of a powerCut. An expectation rule also holds from time period zero for all current hospitals which states that *redLight* will be true within 2 time periods of *turnOff* occurring.

The *addHospital* event initiates a new hospital in the *support* agent, while the *emergency* event initiates a siren at all hospitals.

In [4]:
% File: lights.pl
holdsAt(Agent, lightOn, T):- holdsAt(Agent, greenLight, T), holdsAt(support, applyGeneral(Agent), T).
initially(Agent, greenLight):- holdsAt(support, applyGeneral(Agent), 0).

terminates(Agent, turnOff, greenLight, T):- holdsAt(support, applyGeneral(Agent), T).
terminates("Otago", powerCut, greenLight, T).

initially("Otago", exp_rule(happ(powerCut), within(redLight, 3), dependent, "Within 3 time periods of a power cut, there will be a red light")).

initially(Actor, exp_rule(happ(turnOff), within(redLight, 2), dependent, "Within 2 time periods of turnOff, there will be a red light")):- 
holdsAt(support, applyGeneral(Agent), 0).

initiates(support, addHospital(Hospital), hospital(Hospital), _).

initiates(Agent, emergency, siren, T):- holdsAt(support, applyGeneral(Agent), T).

Our narrative states that a *turnOff* event occurs for Christchurch only at time period 1, and at time period 2, Hamilton is added as a new hospital. A *powerCut* event and an emergency event, both at time period 3, and a *turnOff* event at time period 4, occur for all current hospitals. 

In [5]:
% File: narrative.pl
happensAtNarrative("Christchurch":turnOff, 1).
happensAtNarrative(support:addHospital("Hamilton"),2).
happensAtNarrative(general:powerCut, 3).
happensAtNarrative(general:emergency, 3).
happensAtNarrative(general:turnOff, 4).


The *greenLight* fluent has been inherited by each of the hospitals that exist at time period zero.

In [6]:
?- run(10).
?- holdsAt(Actor, greenLight, 0).

true.
Actor = b'Otago' ;
Actor = b'Christchurch' ;
Actor = b'Wellington' ;
Actor = b'Auckland' .

As the state constraint involving *greenLight* and *lightOn* has been inherited by all of the current hospitals, *lightOn* also holds for each of them.

In [7]:
?- holdsAt(Actor, lightOn, 0).

Actor = b'Otago' ;
Actor = b'Christchurch' ;
Actor = b'Wellington' ;
Actor = b'Auckland' .

*lightOn* continues to hold at time period one, when the *turnOff* event occurs for Christchurch. This event triggers the expectation that *redLight* will hold within 2 time periods for Christchurch only.

In [8]:
?- holdsAt(Actor, lightOn, 1).
?- happensAtNarrative(Actor:Event, 1).
?- holdsAt(Hospital, exp(Result,Status,Message),1).

Actor = b'Otago' ;
Actor = b'Christchurch' ;
Actor = b'Wellington' ;
Actor = b'Auckland' .
Actor = b'Christchurch', Event = turnOff .
Hospital = b'Christchurch', Result = within(redLight, 2), Status = dependent, Message = b'Within 2 time periods of turnOff, there will be a red light' .

At time period 2, *lightOn* no longer holds for Christchurch, as *greenLight* has been terminated by the *turnOff* event. We add the new hospital.

In [9]:
?- holdsAt(Actor, lightOn, 2).
?- happensAtNarrative(Actor:Event, 2).

Actor = b'Otago' ;
Actor = b'Wellington' ;
Actor = b'Auckland' .
Actor = support, Event = addHospital(b'Hamilton') .

At time period 3, the emergency and the power cut occur for all five hospitals. This triggers an expectation that *redLight* will hold within 3 time periods for Otago only.

In [10]:
?- happensAt(Agent, Event, 3).
?- holdsAt(Hospital, exp(Result,Status,Message),3).

Agent = b'Otago', Event = powerCut ;
Agent = b'Otago', Event = emergency ;
Agent = b'Christchurch', Event = powerCut ;
Agent = b'Christchurch', Event = emergency ;
Agent = b'Wellington', Event = powerCut ;
Agent = b'Wellington', Event = emergency ;
Agent = b'Auckland', Event = powerCut ;
Agent = b'Auckland', Event = emergency ;
Agent = b'Hamilton', Event = powerCut ;
Agent = b'Hamilton', Event = emergency ;
Agent = general, Event = powerCut ;
Agent = general, Event = emergency .
Hospital = b'Otago', Result = within(redLight, 3), Status = dependent, Message = b'Within 3 time periods of a power cut, there will be a red light' .

At time period four, *lightOn* no longer holds for Otago, as it was terminated by the *powerCut* event. As Hamilton was added as a new hospital after T = 0, *lightOn* also does not hold for this hospital.

In [11]:
?- holdsAt(Actor, lightOn, 4).

Actor = b'Wellington' ;
Actor = b'Auckland' .

As *siren* is caused by an *initates* clause rather than an *initally* predicate, *siren* holds for Hamilton and the other hospitals. The *turnOff* event occurs at time period 4.

In [12]:
?- holdsAt(Actor, siren, 4).
?- happensAt(general, Event, 4).

Actor = b'Otago' ;
Actor = b'Christchurch' ;
Actor = b'Wellington' ;
Actor = b'Auckland' ;
Actor = b'Hamilton' .
Event = turnOff .

At time period 5, *lightOn* does not hold for any of the hospitals, as it has been terminated for Wellington and Auckland by the *turnOff* event.

In [13]:
?- holdsAt(Actor, lightOn, 5).

false.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>